In [67]:
import json
import re
import os

In [68]:
def longest_common_substring(s1, s2):
    m = [[0] * (1 + len(s2)) for i in range(1 + len(s1))]
    longest, x_longest = 0, 0
    for x in range(1, 1 + len(s1)):
        for y in range(1, 1 + len(s2)):
            if s1[x - 1] == s2[y - 1]:
                m[x][y] = m[x - 1][y - 1] + 1
                if m[x][y] > longest:
                    longest = m[x][y]
                    x_longest = x
            else:
                m[x][y] = 0
    return s1[x_longest - longest: x_longest]



def evaluate_results(results_filepath):
    data_final = []
    with open(results_filepath, 'r') as f:
        data = json.load(f)
    data_final += data

    i = 1
    p, r = 0, 0
    ac = 0
    for item in data_final:
        
        pred = item['pred']
        res = ""
        for char in str(pred[-200:]).split("equence")[-1]:
            if char.isupper():
                res += char # get all uppercase letters as brick names
        if len(res)>=2:
            if res[0] == res[1]:
                res = res[1:] # remove the first letter if it is repeated
        if item['label'][-1] != res[-1]: 
            res += item['label'][-1] # add the last letter of the label (target brick) if it is not already in the prediction
        if res == item['label']: #or res == item['label'][:-1]: #or res[:-1]==item['label']:
            ac += 1
        lcs = longest_common_substring(res,item['label'] )
        r += len(lcs)/len(item['label'])
        if len(res)==0:
            continue
        p += len(lcs)/len(res)
        i += 1
        #ac = 0

    acc = ac/i
    print('acc', acc)
    print(p/i)
    print(r/i)
# models = [
#     'gpt-3.5-turbo',
#     'gpt-4o-mini',
#     'gpt-4o-2024-08-06'
# ]

# for model_name in models:
#     examples_data_dir = '../data/brick_1D_50'
#     results_file_path = os.path.join(examples_data_dir, f'results_{model_name}.json')
#     print(model_name)
#     evaluate_results(results_filepath=results_file_path)

results_dir = '../data/results'
# iterate over json files in the results directory
for file in os.listdir(results_dir):
    if file.endswith('.json'):
        results_file_path = os.path.join(results_dir, file)
        print(file)
        evaluate_results(results_filepath=results_file_path)


2D_text_gpt-4o-mini.json
acc 0.0
0.2952617644345958
0.6701680672268907
1D_text_gpt-3.5-turbo.json
acc 0.5098039215686274
0.7888422035480859
0.6956115779645191
2D_image_color_gpt-4o-mini.json
acc 0.0
0.24341283434117159
0.4310457516339868
1D_text_gpt-4o-mini.json
acc 0.0196078431372549
0.42566358751024563
0.6256238859180036
1D_text_gpt-4o-2024-08-06.json
acc 0.29411764705882354
0.8586288317603191
0.9638344226579522
2D_text_gpt-4o-2024-08-06.json
acc 0.0784313725490196
0.5892138380278796
0.796545284780579
2D_image_color_gpt-4o-2024-08-06.json
acc 0.0
0.27939084934167785
0.6397759103641456
1D_image_color_gpt-4o-2024-08-06.json
acc 0.19607843137254902
0.8103354162177691
0.9732026143790851
1D_image_color_gpt-4o-mini.json
acc 0.45098039215686275
0.652373878844467
0.6411149308208131
2D_text_gpt-3.5-turbo.json
acc 0.0784313725490196
0.25438501009435266
0.45924369747899146


In [128]:
# results_filepath = '../data/results/1D_image_color_gpt-4o-2024-08-06.json'
# results_filepath = '../data/results/1D_image_color_gpt-4o-mini.json'
# results_filepath = '../data/results/1D_text_gpt-3.5-turbo.json'
# results_filepath = '../data/results/1D_text_gpt-4o-2024-08-06.json'
# results_filepath = '../data/results/1D_text_gpt-4o-mini.json'
# results_filepath = '../data/results/2D_image_color_gpt-4o-2024-08-06.json'
# results_filepath = '../data/results/2D_image_color_gpt-4o-mini.json'
# results_filepath = '../data/results/2D_text_gpt-3.5-turbo.json'
# results_filepath = '../data/results/2D_text_gpt-4o-2024-08-06.json'
# results_filepath = '../data/results/2D_text_gpt-4o-mini.json'


data_final = []
with open(results_filepath, 'r') as f:
    data = json.load(f)
data_final += data

i = 1
p, r = 0, 0
ac = 0
for i, item in enumerate(data_final):
    pred = item['pred']
    res = ""

    # pattern = r": *([A-Z, ]+)\." # match the brick names between : and .
    # pattern = r"(?s): *([^:]*[A-Z, ]+)\." # match the brick names between : and .
    # pattern = r": *([A-Za-z, ]+)\."
    # pattern = r": *([A-Za-z, ]+)(\.|$)"
    # pattern2 = r"(?:[:.]) *([A-Za-z, ()*]+)(\.|$)"
    pattern = r': *([A-Z](?![a-z])(?:, *[A-Z](?![a-z]))*)'
    pattern2 = r'(?:[:.]) *([A-Z](?![a-z])(?:, *[A-Z](?![a-z]))*)'
    pattern3 = r'\b[A-Z]\b|(?<![A-Za-z])[A-Z](?![a-z])'

    match = re.search(pattern, item['pred'][-200:])
    if not match:
        match = re.search(pattern2, item['pred'][-200:])
    if not match:
        match = re.search(pattern3, item['pred'][-200:])
        
    if match:
        res = match.group(0)
        res = ''.join(list(filter(lambda x: x.isupper(), res)))
        if len(res)>=2:
            if res[0] == res[1]:
                res = res[1:] # remove the first letter if it is repeated
        if item['label'][-1] != res[-1]: 
            res += item['label'][-1] # add the last letter of the label (target brick) if it is not already in the prediction
        if res == item['label']:
            # print('correct')
            ac += 1
        else:
            # print(item['pred'])
            # print(res)
            # print(item['label'])
            print(f'{i} {item["label"]} \t {res} \t {match.group(0)}')
            print([item['pred'][-200:]])
        lcs = longest_common_substring(res,item['label'] )
        len(lcs)/len(item['label'])
        len(lcs)/len(res)
        if len(res)==0:
            pass
        r += len(lcs)/len(item['label'])
        if len(res)==0:
            continue
        p += len(lcs)/len(res)
    else:
        print(f'no answer found in {item["pred"][-200:]}')
        pass

acc = ac/len(data_final)
print('acc', acc)
print(p/len(data_final))
print(r/len(data_final))

1 FWRC 	 RC 	 : R, C
['f brick C, so we need to remove brick R first.\n2. After removing brick R, we can then grab brick C.\n\nThus, the sequence of bricks to be removed to get brick C is: R, C. \n\nSo the final answer is: R, C.']
2 RFJCZ 	 JCZ 	 : J, C
['o remove brick C next.\n4. Now that brick C is removed, we can finally access brick Z.\n\nThus, the sequence of bricks to be removed from top to bottom to get brick Z is: J, C. \n\nSo, the answer is: J, C.']
3 IGF 	 GF 	 : G, F
[' ignore R for the purpose of grabbing F.\n5. The only brick we need to remove to access brick F is brick G.\n\nThus, the sequence of bricks to remove to get to brick F is: G, F.\n\nSo, the answer is: G, F.']
4 SUZCQL 	 CQL 	 : C, Q
[' N, and M is on top of E, but we are focused on L.\n8. Finally, we can access L after removing Q and C.\n\nThus, the sequence of bricks to be removed to get to brick L is: C, Q.\n\nSo, the answer is: C, Q.']
5 SMRHNQZ 	 FLIONRQZ 	 : F, L, I, O, N, R, Q
['ter removing all the nece

In [90]:
import re

def extract_uppercase(text):
    # Define the pattern to match uppercase letters after a colon or standalone uppercase letters
    pattern = r'(?:: *([A-Z](?![a-z])(?:, *[A-Z](?![a-z]))*))|([A-Z](?![a-z]))'
    
    # Find all matches using the regex pattern
    matches = re.findall(pattern, text)
    
    # Extract the non-empty matches from the tuples returned by re.findall
    result = [match[0] if match[0] else match[1] for match in matches]
    
    # Join the matches with a comma and space for the final result
    return ', '.join(result)

# Example sentences
sentence1 = "So, you only need to grab brick F."
sentence2 = "To get brick I, remove the bricks in this order: N, O, Q, F, L."

# Extract and print the results
result1 = extract_uppercase(sentence1)
print(result1)  # Output: "F"

result2 = extract_uppercase(sentence2)
print(result2)  # Output: "N, O, Q, F, L"

F
I, N, O, Q, F, L


In [56]:
sentence = "To get brick L, you need to remove the bricks above it in this order: F, P, E, I. Then you can take brick L."
# pattern = r"(?<=:\s*)([A-Z, ]+)(?=\.)"
pattern = r": *([A-Z, ]+)\." # match the brick names between : and .

result = ''
match = re.search(pattern, sentence)
if match:
    result = match.group(1)
    result = ''.join(list(filter(lambda x: x.isupper(), result)))
print(result)

FPEI


'FPEI'

In [3]:

i = 1
p, r = 0, 0
ac = 0
for item in data_final:
    pred = item['pred']
    text = str(pred[-200:])
    

    text_list = text.split("equence")
    #print(text_list)
    uppercase_letters = ""
    for char in text_list[-1]:
        if char.isupper():
            uppercase_letters += char
    if len(uppercase_letters)>=2:
        if uppercase_letters[0] == uppercase_letters[1]:
            uppercase_letters = uppercase_letters[1:]
    # print(uppercase_letters)
    # print(item['label'])
    res = uppercase_letters
    if res == item['label']: #or res == item['label'][:-1]: #or res[:-1]==item['label']:
        ac += 1
    lcs = longest_common_substring(res,item['label'] )
    r += len(lcs)/len(item['label'])
    if len(res)==0:
        continue
    p += len(lcs)/len(res)
    i += 1
    #ac = 0

acc = ac/i
print('acc', acc)
print(p/i)
print(r/i)

acc 0.0
0.0
0.0


In [24]:
import re

text = """To get brick Q, you need to follow these steps:

1. Remove brick G.
2. Remove brick U.
3. Remove brick P.
4. Remove brick X.
5. Remove brick Z.
6. Remove brick R.
7. Remove brick C.
8. Finally, grab brick Q.

So the sequence of actions is: remove G, remove U, remove P, remove X, remove Z, remove R, remove C, grab Q."""

pattern = r": *([A-Z, ]+)\."

match = re.search(pattern, text)
if match:
    result = match.group(1).strip()
    print(result)  # Output: "G, U, P, X, Z, R, C, Q"

In [26]:
import re

text = """To get brick Q, you need to follow these steps:
1. Remove brick G
2. Remove brick U
3. Remove brick P.
4. Remove brick X.
5. Remove brick Z.
6. Remove brick R.
7. Remove brick C.
8. Finally, grab brick Q.

So the sequence of actions is: remove G, remove U, remove P, remove X, remove Z, remove R, remove C, grab Q."""

pattern = r": *([A-Za-z, ]+)\."

match = re.search(pattern, text)
if match:
    result = match.group(1).strip()
    print(result)  # Output: "remove G, remove U, remove P, remove X, remove Z, remove R, remove C, grab Q"


remove G, remove U, remove P, remove X, remove Z, remove R, remove C, grab Q
